In [ ]:
import dataframeWithMetadata as dfmd
import dataAgglomeration as da
import os
from matplotlib import pyplot as plt
import pandas as pd
from matplotlib import rcParams
rcParams["figure.dpi"] = 400

In [ ]:
def plot_density_ratio_dframe(dframe):
    """Plots the temporal evolution of the L-INFINITY velocity error norm for each mesh resolution and density ratio."""
    
    timeSchemes = dframe.index.get_level_values("timeSchemes").unique()
    fluxSchemes = dframe.index.get_level_values("fluxSchemes").unique()
    if len(fluxSchemes) >1:
        fluxSchemes = ["upwind", "cubic","limitedLinearV","linear","LUST","MUSCL", "QUICK", "SuperBee", "vanLeer"]
    resolutions = dframe.index.get_level_values("resolution").unique()
    v = 0.01
    
    for resolution in resolutions:
        for timeScheme in timeSchemes:
            for fluxScheme in fluxSchemes:
                if (timeScheme == "Euler") | ((timeScheme == "CrankNicolson") & (fluxScheme == "upwind")):
                    df_subset = dframe.loc[resolution,fluxScheme,timeScheme]
                    plt.plot(df_subset["time"], df_subset["LinfU"] / v, 
                         label=" %s+%s;N%s" % (timeScheme, fluxScheme, resolution))
    plt.ylabel(r"$L_{\infty}(\frac{\|\mathbf{v}_c^e - \mathbf{v}_c^n\|}{\|\mathbf{v}_c^e\|})$")#"$L_{\infty}(\|\mathbf{v}_c^e - \mathbf{v}_c^n\|)$")
    plt.xlabel("Time in seconds")
    plt.yscale('log')
    if dframe["LinfU"].max() > 1e3:
        plt.ylim(bottom=1e-13,top=1e3)
    plt.xlim(-0.005,0.155)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, shadow=True, ncol=2)

In [ ]:
agglomerator = da.data_agglomerator("mercuryAirDropletTranslation.parameter","LinfU.dat", 
                                     directory_pattern="isoAdv-mercuryAirDropletTranslation_00000_templateCase")
agglomerator.show_failed_variations()
agglomerator.write_agglomerated_study_data("isoAdv-mercuryAirDropletTranslation.csv")
dframe_isoAdv = agglomerator.study_dataframe()
dframe_isoAdv = dframe_isoAdv.sort_index()

agglomerator = da.data_agglomerator("mercuryAirDropletTranslation.parameter","LinfU.dat", 
                                     directory_pattern="isoRho-mercuryAirDropletTranslation_00000_templateCase")
agglomerator.show_failed_variations()
agglomerator.write_agglomerated_study_data("isoRho-mercuryAirDropletTranslation.csv")
dframe_isoRho = agglomerator.study_dataframe()
dframe_isoRho = dframe_isoRho.sort_index()

In [ ]:
dframe_isoAdv_Euler_upwind = dframe_isoAdv.loc[(dframe_isoAdv.index.get_level_values('timeSchemes') == 'Euler') & (dframe_isoAdv.index.get_level_values('fluxSchemes') == 'upwind')]
plot_density_ratio_dframe(dframe_isoAdv_Euler_upwind)

In [ ]:
dframe_isoRho_Euler_upwind = dframe_isoRho.loc[(dframe_isoRho.index.get_level_values('timeSchemes') == 'Euler') & (dframe_isoRho.index.get_level_values('fluxSchemes') == 'upwind')]
plot_density_ratio_dframe(dframe_isoRho_Euler_upwind)

In [ ]:
dframe_isoAdv_Euler_N64_CN = dframe_isoAdv.loc[((dframe_isoAdv.index.get_level_values('timeSchemes') == 'Euler') & (dframe_isoAdv.index.get_level_values('resolution') == 64)) 
                                            | ((dframe_isoAdv.index.get_level_values('timeSchemes') == 'CrankNicolson') & (dframe_isoAdv.index.get_level_values('fluxSchemes') == 'upwind')& (dframe_isoAdv.index.get_level_values('resolution') == 64))]                                          

plot_density_ratio_dframe(dframe_isoAdv_Euler_N64_CN)